In [16]:
# Import library
import pandas as pd

# Ignore warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df = pd.read_csv('D:\\Study\\DataScience\\Projects\\AI-Advertising\\SentimentAnalysis\\reviews.csv', sep=',')

In [4]:
# Select relavant columns
review_df = df[['text','customer_sentiment']]

In [5]:
# Filter out neutral reviews
review_df = review_df[review_df['customer_sentiment'] != 'neutral']

In [6]:
# Convert the sentiment column to numeric
sentiment_label = review_df['customer_sentiment'].factorize()

In [7]:
sentiment_label 

(array([0, 1, 1, ..., 0, 1, 1], dtype=int64),
 Index(['positive', 'negative'], dtype='object'))

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

review = review_df['text'].values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review)

vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(review)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [9]:
print(review[0])
print(encoded_docs[0])

@VirginAmerica plus you've added commercials to the experience... tacky.
[103, 575, 530, 1287, 2416, 1, 2, 177]


In [10]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  103  575  530 1287
 2416 

In [11]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))

model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           423488    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)              

In [12]:
history = model.fit(padded_sequence,sentiment_label[0], validation_split=0.2, epochs=10, batch_size=32)

Train on 9232 samples, validate on 2309 samples
Epoch 1/10
9232/9232 [==============================] - 63s 7ms/sample - loss: 0.4488 - acc: 0.8135 - val_loss: 0.2364 - val_acc: 0.9004
Epoch 2/10
9232/9232 [==============================] - 49s 5ms/sample - loss: 0.2591 - acc: 0.8995 - val_loss: 0.1947 - val_acc: 0.9238
Epoch 3/10
9232/9232 [==============================] - 48s 5ms/sample - loss: 0.1951 - acc: 0.9272 - val_loss: 0.1827 - val_acc: 0.9298
Epoch 4/10
9232/9232 [==============================] - 49s 5ms/sample - loss: 0.1629 - acc: 0.9394 - val_loss: 0.1751 - val_acc: 0.9355
Epoch 5/10
9232/9232 [==============================] - 47s 5ms/sample - loss: 0.1328 - acc: 0.9521 - val_loss: 0.1826 - val_acc: 0.9294
Epoch 6/10
9232/9232 [==============================] - 48s 5ms/sample - loss: 0.1181 - acc: 0.9581 - val_loss: 0.1948 - val_acc: 0.9355
Epoch 7/10
9232/9232 [==============================] - 47s 5ms/sample - loss: 0.1036 - acc: 0.9638 - val_loss: 0.2072 - val_acc: 

In [13]:
# Save model to disk
model_name = 'D:\\Study\\DataScience\\Projects\\AI-Advertising\\SentimentAnalysis\\customer_sentiment.h5'

model.save(model_name)

In [14]:
# Load a compiled model
from tensorflow.keras.models import load_model

sentiment_model = load_model(model_name)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
test_word = "This is the worst part."

tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)

prediction = int(sentiment_model.predict(tw).round().item())

# Print the sentiment
print('This is a {} sentiment.'.format(sentiment_label[1][prediction]))

This is a negative sentiment.
